In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from openpyxl.drawing.image import Image as ExcelImage
from openpyxl.utils import get_column_letter
from openpyxl import load_workbook
import pandas as pd
import os

from config_parameters import (
    ANALYSIS_TICKER_INTERVAL_FOLDER,
    SIMULATION_DATA_DICT
)


def generate_and_insert_plot(wb, summary_df, sheet_name, simulation_data_specific,ticker='TICKER', strategies=["0.0"], intervals="1d", row=20, col=15):
    """
    Generates a chart of account value over time and inserts it into a worksheet in the workbook.

    Parameters:
        wb (Workbook): The Excel workbook to insert the chart into.
        summary_df (pd.DataFrame): DataFrame containing strategy metadata.
        sheet_name (str): Name of the Excel sheet.
        ticker (str): Asset ticker.
        strategies (list): List of strategy IDs as strings.
        intervals (list): List of intervals (e.g., ["1d", "4h"]).
        row (int): Starting row for chart placement.
        col (int): Starting column for chart placement.
    """
    current_row = row
    for interval in intervals:
        plt.figure(figsize=(12, 6))
        for strategy in strategies:
            history_path = f"{simulation_data_specific}/{ticker} - strategy {strategy}, interval {interval} - history.xlsx"
            history_df = pd.read_excel(history_path)
            strategy_description = summary_df[summary_df['Strategy'] == float(strategy)].iloc[0]['Description']

            if 'timestamp' in history_df.columns and 'account_value' in history_df.columns:
                if float(strategy) == 0.0:
                    plt.plot(
                        history_df['timestamp'],
                        history_df['account_value'],
                        label=f"Strategy {strategy} - {strategy_description}",
                        color='black',
                        linewidth=1.5,
                        alpha=0.3
                    )
                else:
                    plt.plot(
                        history_df['timestamp'],
                        history_df['account_value'],
                        label=f"Strategy {strategy} - {strategy_description}",
                        linewidth=1.5
                    )

        # Save the chart
        os.makedirs('Charts', exist_ok=True)
        image_path = os.path.join('Charts', f'{ticker}-{interval}.png')
        plt.title(f'Account Value Over Time\nTicker: {ticker}, Interval: {interval}, Start Date: {start_date}')
        plt.xlabel('Date')
        plt.ylabel('Account Value')
        plt.grid(True)
        plt.legend()
        ax = plt.gca()
        ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'{int(x):,}'))
        plt.tight_layout()
        plt.savefig(image_path)
        plt.close()

        # === 2. Insert the chart into Excel sheet ===
        if sheet_name not in wb.sheetnames:
            wb.create_sheet(title=sheet_name)

        ws = wb[sheet_name]
        img = ExcelImage(image_path)
        col_letter = get_column_letter(col)
        anchor_cell = f"{col_letter}{current_row}"
        img.anchor = anchor_cell
        ws.add_image(img)
        current_row += 30  # move further down to avoid overlapping charts


def generate_charts_for_tickers(start_date, end_date, strategies, intervals):
# === Setup ===
    simulation_data_dir = SIMULATION_DATA_DICT
    simulation_data_specific = os.path.join(simulation_data_dir, f"{start_date}_to_{end_date}")



    analysis_dir = ANALYSIS_TICKER_INTERVAL_FOLDER
    os.makedirs(analysis_dir, exist_ok=True)
    sheet_output = os.path.join(analysis_dir, f"{start_date}_to_{end_date}.xlsx")

    # Load existing workbook
    wb = load_workbook(sheet_output)

    # Load summary sheet
    simulation_sheet_output = f'Simulations_summaries- {start_date}.xlsx'
    summary_df = pd.read_excel(simulation_sheet_output)

    tickers = summary_df['Ticker'].unique()
    for ticker in tickers:
        generate_and_insert_plot(
            wb,
            summary_df,
            sheet_name=ticker,
            simulation_data_specific,
            ticker=ticker,
            strategies=strategies,
            intervals=intervals,
            row=1,
            col=16
        )

    # Save final workbook
    wb.save(sheet_output)


start_date = "2024-01-01"
end_date = "2025-02-01"
period_id = '3'
intervals = ['1w', '3d','2d', '1d', '12h', '6h', '4h', '2h', '1h', '30m', '15m']
strategies = ['0.0', '0.1', '0.3', '0.4']
intervals_to_charts = ["1d", "12h", "4h"]